In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import math
from datetime import datetime,timedelta

In [89]:
def get_newscontent(l_url) :
    news_content = []
    text_save = []
    logi_url = 'http://www.klnews.co.kr/news/'+l_url
    print(logi_url)
    get_req = requests.get(logi_url)
    get_soup = BeautifulSoup(get_req.content, 'html.parser')
    
    try:
        sub_title = get_soup.select('td.subartView-sub-title')[0].text.replace('“' , '').replace('”','') #부제목
    except:
        sub_title =''
    news_content.append(sub_title)
        

    
    
    try:
        _text = get_soup.select('div >p')
        for i in range(0,len(_text)-1) : #내용저장
            text_save.append(get_soup.select('div >p')[i].text)
        text_save = ''.join(text_save)
    except :
        text_save =''
    news_content.append(text_save)
    
    return news_content
    
    
    


In [ ]:
page = 1
url = 'http://www.klnews.co.kr/news/articleList.html?page='+str(page)+'&total=57252&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type='
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
maxpage = soup.select('font')[1].text
maxpage= math.ceil(int(maxpage[6:-2].replace(',',"").strip())/20)

news_box =[]
print('총페이지',maxpage)

while page < maxpage +1 :
    print('현재페에지', page)
    a = 0
    for i in range(len(soup.select('td.ArtList_Title > a'))):
        
        try :
            
            urls = soup.select('td.ArtList_Title > a')[i]['href']
            news_content2 = get_newscontent(urls)
            data = soup.select('td.View_SmFont.FontEng')[i].text
            classify = soup.select('td.ArtList_Title > font')[i].text.replace('[',"").replace(']',"")
            title = soup.select('td.ArtList_Title > a')[i].text
            sub_title= news_content2[0]
            contents = news_content2[1]
            news_total = [data , classify, title,sub_title,contents]
            news_box.append(news_total)
                
        
        except Exception as e:
            print(e)
            
            
            
    page +=1
    url = 'http://www.klnews.co.kr/news/articleList.html?page='+str(page)+'&total=57252&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type='
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    

총페이지 2863
현재페에지 1
http://www.klnews.co.kr/news/articleView.html?idxno=121605
http://www.klnews.co.kr/news/articleView.html?idxno=121604
http://www.klnews.co.kr/news/articleView.html?idxno=121603
http://www.klnews.co.kr/news/articleView.html?idxno=121602
http://www.klnews.co.kr/news/articleView.html?idxno=121601
http://www.klnews.co.kr/news/articleView.html?idxno=121600
http://www.klnews.co.kr/news/articleView.html?idxno=121599
http://www.klnews.co.kr/news/articleView.html?idxno=121598
http://www.klnews.co.kr/news/articleView.html?idxno=121597
http://www.klnews.co.kr/news/articleView.html?idxno=121596
http://www.klnews.co.kr/news/articleView.html?idxno=121595
http://www.klnews.co.kr/news/articleView.html?idxno=121593
http://www.klnews.co.kr/news/articleView.html?idxno=121594
http://www.klnews.co.kr/news/articleView.html?idxno=121591
http://www.klnews.co.kr/news/articleView.html?idxno=121592
http://www.klnews.co.kr/news/articleView.html?idxno=121590
http://www.klnews.co.kr/news/articleVi

http://www.klnews.co.kr/news/articleView.html?idxno=121465
http://www.klnews.co.kr/news/articleView.html?idxno=121464
현재페에지 8
http://www.klnews.co.kr/news/articleView.html?idxno=121463
http://www.klnews.co.kr/news/articleView.html?idxno=121462
http://www.klnews.co.kr/news/articleView.html?idxno=121460
http://www.klnews.co.kr/news/articleView.html?idxno=121459
http://www.klnews.co.kr/news/articleView.html?idxno=121458
http://www.klnews.co.kr/news/articleView.html?idxno=121457
http://www.klnews.co.kr/news/articleView.html?idxno=121456
http://www.klnews.co.kr/news/articleView.html?idxno=121454
http://www.klnews.co.kr/news/articleView.html?idxno=121455
http://www.klnews.co.kr/news/articleView.html?idxno=121453
http://www.klnews.co.kr/news/articleView.html?idxno=121452
http://www.klnews.co.kr/news/articleView.html?idxno=121451
http://www.klnews.co.kr/news/articleView.html?idxno=121450
http://www.klnews.co.kr/news/articleView.html?idxno=121448
http://www.klnews.co.kr/news/articleView.html?id

http://www.klnews.co.kr/news/articleView.html?idxno=121322
현재페에지 15
http://www.klnews.co.kr/news/articleView.html?idxno=121321
http://www.klnews.co.kr/news/articleView.html?idxno=121320
http://www.klnews.co.kr/news/articleView.html?idxno=121319
http://www.klnews.co.kr/news/articleView.html?idxno=121318
http://www.klnews.co.kr/news/articleView.html?idxno=121317
http://www.klnews.co.kr/news/articleView.html?idxno=121316
http://www.klnews.co.kr/news/articleView.html?idxno=121315
http://www.klnews.co.kr/news/articleView.html?idxno=121313
http://www.klnews.co.kr/news/articleView.html?idxno=121314
http://www.klnews.co.kr/news/articleView.html?idxno=121312
http://www.klnews.co.kr/news/articleView.html?idxno=121311
http://www.klnews.co.kr/news/articleView.html?idxno=121310
http://www.klnews.co.kr/news/articleView.html?idxno=121309
http://www.klnews.co.kr/news/articleView.html?idxno=121308
http://www.klnews.co.kr/news/articleView.html?idxno=121307
http://www.klnews.co.kr/news/articleView.html?i

http://www.klnews.co.kr/news/articleView.html?idxno=121181
http://www.klnews.co.kr/news/articleView.html?idxno=121180
http://www.klnews.co.kr/news/articleView.html?idxno=121179
http://www.klnews.co.kr/news/articleView.html?idxno=121178
현재페에지 22
http://www.klnews.co.kr/news/articleView.html?idxno=121177
http://www.klnews.co.kr/news/articleView.html?idxno=121176
http://www.klnews.co.kr/news/articleView.html?idxno=121174
http://www.klnews.co.kr/news/articleView.html?idxno=121173
http://www.klnews.co.kr/news/articleView.html?idxno=121172
http://www.klnews.co.kr/news/articleView.html?idxno=121171
http://www.klnews.co.kr/news/articleView.html?idxno=121170
http://www.klnews.co.kr/news/articleView.html?idxno=121169
http://www.klnews.co.kr/news/articleView.html?idxno=121168
http://www.klnews.co.kr/news/articleView.html?idxno=121167
http://www.klnews.co.kr/news/articleView.html?idxno=121166
http://www.klnews.co.kr/news/articleView.html?idxno=121165
http://www.klnews.co.kr/news/articleView.html?i

http://www.klnews.co.kr/news/articleView.html?idxno=121041
http://www.klnews.co.kr/news/articleView.html?idxno=121040
http://www.klnews.co.kr/news/articleView.html?idxno=121039
http://www.klnews.co.kr/news/articleView.html?idxno=121038
http://www.klnews.co.kr/news/articleView.html?idxno=121037
http://www.klnews.co.kr/news/articleView.html?idxno=121036
http://www.klnews.co.kr/news/articleView.html?idxno=121035
현재페에지 29
http://www.klnews.co.kr/news/articleView.html?idxno=121034
http://www.klnews.co.kr/news/articleView.html?idxno=121033
http://www.klnews.co.kr/news/articleView.html?idxno=121031
http://www.klnews.co.kr/news/articleView.html?idxno=121032
http://www.klnews.co.kr/news/articleView.html?idxno=121030
http://www.klnews.co.kr/news/articleView.html?idxno=121029
http://www.klnews.co.kr/news/articleView.html?idxno=121028
http://www.klnews.co.kr/news/articleView.html?idxno=121027
http://www.klnews.co.kr/news/articleView.html?idxno=121026
http://www.klnews.co.kr/news/articleView.html?i

...

http://www.klnews.co.kr/news/articleView.html?idxno=115866
http://www.klnews.co.kr/news/articleView.html?idxno=115865
현재페에지 279
http://www.klnews.co.kr/news/articleView.html?idxno=115864
http://www.klnews.co.kr/news/articleView.html?idxno=115863
http://www.klnews.co.kr/news/articleView.html?idxno=115862
http://www.klnews.co.kr/news/articleView.html?idxno=115861
http://www.klnews.co.kr/news/articleView.html?idxno=115860
http://www.klnews.co.kr/news/articleView.html?idxno=115859
http://www.klnews.co.kr/news/articleView.html?idxno=115858
http://www.klnews.co.kr/news/articleView.html?idxno=115857
http://www.klnews.co.kr/news/articleView.html?idxno=115856
http://www.klnews.co.kr/news/articleView.html?idxno=115855
http://www.klnews.co.kr/news/articleView.html?idxno=115854
http://www.klnews.co.kr/news/articleView.html?idxno=115853
http://www.klnews.co.kr/news/articleView.html?idxno=115852
http://www.klnews.co.kr/news/articleView.html?idxno=115851
http://www.klnews.co.kr/news/articleView.html?

http://www.klnews.co.kr/news/articleView.html?idxno=115726
http://www.klnews.co.kr/news/articleView.html?idxno=115725
http://www.klnews.co.kr/news/articleView.html?idxno=115724
현재페에지 286
http://www.klnews.co.kr/news/articleView.html?idxno=115723
http://www.klnews.co.kr/news/articleView.html?idxno=115722
http://www.klnews.co.kr/news/articleView.html?idxno=115721
http://www.klnews.co.kr/news/articleView.html?idxno=115720
http://www.klnews.co.kr/news/articleView.html?idxno=115719
http://www.klnews.co.kr/news/articleView.html?idxno=115718
http://www.klnews.co.kr/news/articleView.html?idxno=115717
http://www.klnews.co.kr/news/articleView.html?idxno=115716
http://www.klnews.co.kr/news/articleView.html?idxno=115715
http://www.klnews.co.kr/news/articleView.html?idxno=115714
http://www.klnews.co.kr/news/articleView.html?idxno=115713
http://www.klnews.co.kr/news/articleView.html?idxno=115712
http://www.klnews.co.kr/news/articleView.html?idxno=115711
http://www.klnews.co.kr/news/articleView.html?

http://www.klnews.co.kr/news/articleView.html?idxno=115583
http://www.klnews.co.kr/news/articleView.html?idxno=115582
현재페에지 293
http://www.klnews.co.kr/news/articleView.html?idxno=115581
http://www.klnews.co.kr/news/articleView.html?idxno=115580
http://www.klnews.co.kr/news/articleView.html?idxno=115579
http://www.klnews.co.kr/news/articleView.html?idxno=115578
http://www.klnews.co.kr/news/articleView.html?idxno=115577
http://www.klnews.co.kr/news/articleView.html?idxno=115576
http://www.klnews.co.kr/news/articleView.html?idxno=115575
http://www.klnews.co.kr/news/articleView.html?idxno=115574
http://www.klnews.co.kr/news/articleView.html?idxno=115573
http://www.klnews.co.kr/news/articleView.html?idxno=115571
http://www.klnews.co.kr/news/articleView.html?idxno=115570
http://www.klnews.co.kr/news/articleView.html?idxno=115569
http://www.klnews.co.kr/news/articleView.html?idxno=115568
http://www.klnews.co.kr/news/articleView.html?idxno=115567
http://www.klnews.co.kr/news/articleView.html?

http://www.klnews.co.kr/news/articleView.html?idxno=115443
http://www.klnews.co.kr/news/articleView.html?idxno=115442
http://www.klnews.co.kr/news/articleView.html?idxno=115441
현재페에지 300
http://www.klnews.co.kr/news/articleView.html?idxno=115440
http://www.klnews.co.kr/news/articleView.html?idxno=115439
http://www.klnews.co.kr/news/articleView.html?idxno=115438
http://www.klnews.co.kr/news/articleView.html?idxno=115437
http://www.klnews.co.kr/news/articleView.html?idxno=115436
http://www.klnews.co.kr/news/articleView.html?idxno=115435
http://www.klnews.co.kr/news/articleView.html?idxno=115434
http://www.klnews.co.kr/news/articleView.html?idxno=115433
http://www.klnews.co.kr/news/articleView.html?idxno=115432
http://www.klnews.co.kr/news/articleView.html?idxno=115431
http://www.klnews.co.kr/news/articleView.html?idxno=115430
http://www.klnews.co.kr/news/articleView.html?idxno=115429
http://www.klnews.co.kr/news/articleView.html?idxno=115428
http://www.klnews.co.kr/news/articleView.html?

http://www.klnews.co.kr/news/articleView.html?idxno=115305
http://www.klnews.co.kr/news/articleView.html?idxno=115304
http://www.klnews.co.kr/news/articleView.html?idxno=115303
http://www.klnews.co.kr/news/articleView.html?idxno=115302
http://www.klnews.co.kr/news/articleView.html?idxno=115301
현재페에지 307
http://www.klnews.co.kr/news/articleView.html?idxno=115300
http://www.klnews.co.kr/news/articleView.html?idxno=115299
http://www.klnews.co.kr/news/articleView.html?idxno=115298
http://www.klnews.co.kr/news/articleView.html?idxno=115297
http://www.klnews.co.kr/news/articleView.html?idxno=115296
http://www.klnews.co.kr/news/articleView.html?idxno=115295
http://www.klnews.co.kr/news/articleView.html?idxno=115294
http://www.klnews.co.kr/news/articleView.html?idxno=115293
http://www.klnews.co.kr/news/articleView.html?idxno=115292
http://www.klnews.co.kr/news/articleView.html?idxno=115291
http://www.klnews.co.kr/news/articleView.html?idxno=115290
http://www.klnews.co.kr/news/articleView.html?

In [93]:
data =pd.DataFrame(news_box)
data.columns = ['data', 'class', 'title','subtitle', 'content']
data.to_excel('./loginews.xlsx', index =False)